# Environment Prepare

In [ ]:
pip install noaa-sdk

In [ ]:
!pip install pyspark py4j

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys    
path_to_module = '/content/drive/MyDrive/2022Spring/6889ls/final'
sys.path.append(path_to_module)

In [ ]:
from noaa_sdk import NOAA
import random

In [ ]:
from pyspark import SparkConf, SparkContext
from datetime import datetime
import numpy as np
conf = SparkConf("local").setAppName("myApp") 
sc = SparkContext(conf = conf)

In [ ]:
from line_prepare_file import line_prepare
from streaming_algo import algo

# Data preprocess

In [ ]:
zipcode_list = ['36043', '85034', '72223', '95837', '80294', '06101', '19808', '20001', '32310', '30002', '96817', '83704', '01020', '70801', '46077', '50047', '66546', '40601', '04330', '21401', '02108', '48864', '55101', '39056', '65043', '59601', '68501', '89403', '03301', '08609', '87509','10001','27007','58501','43085','73132','97209','19050','02841','29201','57701','37201','77001','84101','05702','23225','98101','25301','53201','82601','00601']
states_list = ['Alabama','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District Of Columbia','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming','Puerto Rico']

In [ ]:
def obtain(zipcode):
    raw_data = []
    for i in range(24):
        raw_data.append(NOAA().get_forecasts(zipcode, 'US')[i])

    feature_list = ['number','startTime','endTime','shortForecast','temperature','temperatureUnit','windDirection','windSpeed']
    get_data = []

    for data in raw_data:
        tmp = []
        for fea in feature_list:
            tmp.append(data.get(fea))
        get_data.append(tmp)
    
    with open('/content/drive/MyDrive/2022Spring/6889ls/final/test_data.txt', 'w') as f:
        for item in get_data:
            f.write("%s\n" % item)
    
    line=sc.textFile('/content/drive/MyDrive/2022Spring/6889ls/final/test_data.txt')
    return line

In [ ]:
result=[]

for zipcode in zipcode_list:
  line = obtain(zipcode)
  pspdata = line_prepare(line)
  result.append(algo(pspdata))

output = sc.parallelize(result)

Previous request failed with code 500. Retrying...
Previous Response: {
    "correlationId": "4fe4227",
    "title": "Unexpected Problem",
    "type": "https://api.weather.gov/problems/UnexpectedProblem",
    "status": 500,
    "detail": "An unexpected problem has occurred.",
    "instance": "https://api.weather.gov/requests/4fe4227"
}
Previous request failed with code 500. Retrying...
Previous Response: {
    "correlationId": "5dac3348",
    "title": "Unexpected Problem",
    "type": "https://api.weather.gov/problems/UnexpectedProblem",
    "status": 500,
    "detail": "An unexpected problem has occurred.",
    "instance": "https://api.weather.gov/requests/5dac3348"
}
Previous request failed with code 500. Retrying...
Previous Response: {
    "correlationId": "48ec4c82",
    "title": "Unexpected Problem",
    "type": "https://api.weather.gov/problems/UnexpectedProblem",
    "status": 500,
    "detail": "An unexpected problem has occurred.",
    "instance": "https://api.weather.gov/req

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

df = spark.createDataFrame(output,["Weather","MinTemperature","MaxTemperature","MinWindSpeed","MaxWindSpeed","WindDirection"])
df.write.format('json').save('/content/drive/MyDrive/2022Spring/6889ls/final/test.json')

In [ ]:
df.show()

+--------------------+-----------------+-----------------+-----------------+-----------------+-------------+
|             Weather|MinTemperature(F)|MaxTemperature(F)|MinWindSpeed(mph)|MaxWindSpeed(mph)|WindDirection|
+--------------------+-----------------+-----------------+-----------------+-----------------+-------------+
|        Mostly Clear|             67.0|             89.0|              0.0|              5.0|            W|
|               Sunny|             66.0|             92.0|              0.0|             10.0|          WSW|
|        Partly Sunny|             64.0|             73.0|              5.0|             10.0|            E|
|               Sunny|             52.0|             88.0|              3.0|             13.0|            S|
|        Partly Sunny|             39.0|             48.0|              5.0|             13.0|            N|
| Chance Rain Showers|             49.0|             63.0|              3.0|              9.0|            N|
|Slight Chance Rai.